# Hands-on with Spark (Structured Streaming)

![spark](https://cdn-images-1.medium.com/max/300/1*c8CtvqKJDVUnMoPGujF5fA.png)

In the previous lesson, we learnt about Spark SQL, Dataframes and Pandas API. In this lesson, we will continue with the Structured Streaming.

Structured Streaming is a scalable and fault-tolerant stream processing engine built on the Spark SQL engine. You can express your streaming computation the same way you would express a batch computation on static data. The Spark SQL engine will take care of running it incrementally and continuously and updating the final result as streaming data continues to arrive.

You can use the Dataset/DataFrame API to express streaming aggregations, event-time windows, stream-to-batch joins, etc. The computation is executed on the same optimized Spark SQL engine. Finally, the system ensures end-to-end exactly-once fault-tolerance guarantees through checkpointing and Write-Ahead Logs. In short, Structured Streaming provides **fast, scalable, fault-tolerant, end-to-end exactly-once** stream processing without the user having to reason about streaming.

Internally, by default, Structured Streaming queries are processed using a micro-batch processing engine, which processes data streams as a series of small batch jobs thereby achieving end-to-end latencies as low as 100 milliseconds and exactly-once fault-tolerance guarantees.

## Installing and Initializing Spark

First, like previously, we'll need to install Spark and its dependencies:

1.   Java 8
2.   Apache Spark with Hadoop
3.   Findspark (used to locate the Spark in the system)


In [ ]:
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

# set the options to connect to our Kafka cluster
options = {
    # "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.scram.ScramLoginModule required username="YnJhdmUtZmlzaC0xMTQ2MyQSvwXBuLOQsV1W7YffuC8cDaZcA3fKQwakMhnQGgg" password="MDUxNjc4YzEtYzYxNy00NTE1LWEwNWYtMDBhODRlZmE0OGJm";',
    # "kafka.sasl.mechanism": "SCRAM-SHA-256",
    # "kafka.security.protocol" : "SASL_SSL",
    "kafka.bootstrap.servers": 'localhost:9092',
    "subscribe": 'pizza-orders',
}

In [ ]:
# import findspark
# findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LearnSparkStreaming").getOrCreate()

In [ ]:
spark

## Read and Analyze Kafka stream in "Batch" Mode

Let's start with reading and analyzing our `pizza-orders` kafka topic in the usual "batch" mode of Spark SQL and Dataframes. This is akin to the batch queries we did in the previous lesson. In this case we are taking the messages with the earliest to latest offsets of the topic as a single "batch".

In [ ]:
pizza_df = spark.read.format('kafka')\
    .options(**options)\
    .load()

In [ ]:
pizza_df.printSchema()

In [ ]:
pizza_df.show()

In [ ]:
pizza_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").show()

In [ ]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StringType, IntegerType, LongType, DoubleType, StructType, ArrayType, StructField

In [ ]:
pizza_schema = StructType([
  StructField("pizzaName", StringType()),
  StructField("additionalToppings", ArrayType(StringType())),
])

order_schema = StructType([
  StructField("address", StringType()),
  StructField("id", IntegerType()),
  StructField("name", StringType()),
  StructField("phoneNumber", StringType()),
  StructField("shop", StringType()),
  StructField("cost", DoubleType()),
  StructField("pizzas", ArrayType(pizza_schema)),
  StructField("timestamp", LongType()),
])

In [ ]:
parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value"))

In [ ]:
parsed_df.printSchema()

In [ ]:
parsed_df.show(truncate=False)

We can use _dot notation_ to select the field within a `Struct`:

In [ ]:
parsed_df.select("value.cost").show()

Computing the "total revenue" per shop:

In [ ]:
parsed_df.groupBy("value.shop").sum("value.cost").show(truncate=False)

> 1. Count the no. of orders by shop.
> 2. Compute the avg revenue by shop and sort by highest to lowest.

In [ ]:
from pyspark.sql.functions import min, max

In [ ]:
parsed_df.select(min("timestamp"), max("timestamp")).show(truncate=False)

## Read and Analyze Kafka stream in "Streaming" Mode

The key idea in Structured Streaming is to treat a live data stream as a table that is being continuously appended. This leads to a new stream processing model that is very similar to a batch processing model. You will express your streaming computation as standard batch-like query as on a static table, and Spark runs it as an incremental query on the *unbounded input table*. Let’s understand this model in more detail.

Consider the input data stream as the “Input Table”. Every data item that is arriving on the stream is like a new row being appended to the Input Table.

![concept](https://spark.apache.org/docs/latest/img/structured-streaming-stream-as-a-table.png)

A query on the input will generate the “Result Table”. Every trigger interval (say, every 1 second), new rows get appended to the Input Table, which eventually updates the Result Table. Whenever the result table gets updated, we would want to write the changed result rows to an external sink.

![result table](https://spark.apache.org/docs/latest/img/structured-streaming-model.png)

To illustrate the use of this model, let’s understand the model in context of a word count model. The first lines DataFrame is the input table, and the final wordCounts DataFrame is the result table. Note that the query on streaming lines DataFrame to generate wordCounts is exactly the same as it would be a static DataFrame. However, when this query is started, Spark will continuously check for new data from the socket connection. If there is new data, Spark will run an “incremental” query that combines the previous running counts with the new data to compute updated counts, as shown below.

![example](https://spark.apache.org/docs/latest/img/structured-streaming-example-model.png)




Event-time is the time embedded in the data itself. For many applications, you may want to operate on this event-time.

For example, if you want to get the number of events generated by IoT devices every minute, then you probably want to use the time when the data was generated (that is, event-time in the data), rather than the time Spark receives them.

This event-time is very naturally expressed in this model – each event from the devices is a row in the table, and event-time is a column value in the row. This allows window-based aggregations (e.g. number of events every minute) to be just a special type of grouping and aggregation on the event-time column – each time window is a group and each row can belong to multiple windows/groups. Therefore, such event-time-window-based aggregation queries can be defined consistently on both a static dataset (e.g. from collected device events logs) as well as on a data stream, making the life of the user much easier.

Furthermore, this model naturally handles data that has arrived later than expected based on its event-time. Since Spark is updating the Result Table, it has full control over updating old aggregates when there is late data, as well as cleaning up old aggregates to limit the size of intermediate state data.

In [ ]:
pizza_df = spark.readStream.format('kafka')\
    .options(**options)\
    .load()

In [ ]:
pizza_df.isStreaming

In [ ]:
pizza_df.printSchema()

In [ ]:
parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value"))

In [ ]:
parsed_df.writeStream.format("console").start()

In [ ]:
parsed_df.printSchema()

In [ ]:
query = parsed_df.where("value.cost > 10")

In [ ]:
query = parsed_df \
    .writeStream \
    .format("console") \
    .start()

query.awaitTermination() # stops the script from exiting
# query.isActive
# query.recentProgress
# query.stop()

In [ ]:
query.recentProgress

### In the next few cells, we will provide a complete example of counting the number of pizza shops with orders, as the orders stream in.

In [ ]:
# First check if any streams are active
spark.streams.active

In [ ]:
# If there are any active streams, stop them
for q in spark.streams.active:
    print(f"Stopping query: {q.name}")
    q.stop()

In [ ]:
# Complete example with counting
pizza_df = spark.readStream.format('kafka')\
    .options(**options)\
    .load()

parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value")) #.groupBy("value.shop").count()

shop_counts = parsed_df.groupBy("value.shop").count()

# outputMode("complete") rewrites all aggregated results every trigger — good for full snapshots.
# outputMode("update") - only updated rows are written
query = shop_counts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

In [ ]:
query.stop()

In [ ]:
query.recentProgress

You have come to the end of this exercise.

To delete the Kafka topic `pizza-orders`, use the command below in your terminal:

`./kafka-topics.sh --delete --topic pizza-orders --bootstrap-server localhost:9092`

To exit Kafka in your terminal, type `exit`.